In [2]:
# Split the corpus by the given inputs, downselecting randomly to equal sizes
import sqlite3
import pandas
con = sqlite3.connect('Reddit/Reddit.db')
cur = con.cursor()

# Make all good, all bad sets (both by answer and by view)
reddit_answer_good = pandas.read_sql('select * from data where answer_good is 1', con)
reddit_answer_bad = pandas.read_sql('select * from data where answer_bad is 1', con)
con.close()
reddit_max_size = min(len(reddit_answer_good), len(reddit_answer_bad))

In [3]:
# Split the corpus by the given inputs, downselecting randomly to equal sizes
import sqlite3
import pandas
con = sqlite3.connect('StackOverflow/SO.db')
cur = con.cursor()

# Make all good, all bad sets (both by answer and by view)
so_answer_good = pandas.read_sql('select * from data where answer_good is 1', con)
so_answer_bad = pandas.read_sql('select * from data where answer_bad is 1', con)
con.close()
so_max_size = min(len(so_answer_good), len(so_answer_bad))

In [4]:
max_size = min(so_max_size, reddit_max_size)
reddit_good_sample = reddit_answer_good.sample(n=max_size)
reddit_bad_sample = reddit_answer_bad.sample(n=max_size)
so_good_sample = so_answer_good.sample(n=max_size)
so_bad_sample = so_answer_bad.sample(n=max_size)
print(max_size)

18791


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(pandas.concat([so_good_sample, so_bad_sample, 
                                              reddit_good_sample, reddit_bad_sample]), test_size=0.5)
import pickle
with open('combined_train_test.p', 'wb') as pfile:
    pickle.dump((train, test), pfile)

In [7]:
with open('r_train_so_test.p', 'wb') as pfile:
    pickle.dump((pandas.concat([reddit_good_sample, reddit_bad_sample]), 
                 pandas.concat([so_good_sample, so_bad_sample])), pfile)
with open('so_train_r_test.p', 'wb') as pfile:
    pickle.dump((pandas.concat([so_good_sample, so_bad_sample]), 
                 pandas.concat([reddit_good_sample, reddit_bad_sample])), pfile)

In [8]:
filenames = ['combined_train_test.p', 'r_train_so_test.p', 'so_train_r_test.p',
            'StackOverflow/answer_data.p', 'Reddit/answer_data.p']
for filename in filenames:
    with open(filename, 'rb') as pfile:
        train, test = pickle.load(pfile)
    train = train.assign(index = train.index.values)
    test = test.assign(index = train.index.values)
    with open(filename, 'wb') as pfile:
        pickle.dump((train, test), pfile)